# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

Country/Region     Cases  Deaths    PCases  PDeaths  Cases (+)  \
0               US  33326437  596434  33254998   594319      71439   
1            India  28574350  340702  27894800   325972     679550   
2           Brazil  16803472  469388  16471600   461057     331872   
3           France   5755679  109990   5719877   109518      35802   
4           Turkey   5270299   47882   5235978    47271      34321   
5           Russia   5040390  120604   4995613   118781      44777   
6   United Kingdom   4515778  128075   4496823   128037      18955   
7            Italy   4225163  126342   4213055   126002      12108   
8        Argentina   3884447   79873   3732263    77108     152184   
9          Germany   3701692   89031   3684672    88413      17020   
10           Spain   3693012   80049   3668658    79905      24354   
11        Colombia   3488046   90353   3363061    87747     124985   
12            Iran   2945100   80658   2893218    79741      51882   
13          Poland   2874092   74075   2871371    73682       2721   
14          Mexico   2426822  228362   2411503   223455      15319   

    Deaths (+)  Fatality Rate      Continent  
0         2115            1.8  North America  
1        14730            1.2           Asia  
2         8331            2.8  South America  
3          472            1.9         Europe  
4          611            0.9           Asia  
5         1823            2.4         Europe  
6           38            2.8         Europe  
7          340            3.0         Europe  
8         2765            2.1  South America  
9          618            2.4         Europe  
10         144            2.2         Europe  
11        2606            2.6  South America  
12         917            2.7           Asia  
13         393            2.6         Europe  
14        4907            9.4  North America

In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

{'updated': Timestamp('2021-06-03 00:00:00'),
 'since': Timestamp('2021-05-29 00:00:00'),
 'Cases': 172062878,
 'Deaths': 3699325,
 'Cases (+)': 2215205,
 'Deaths (+)': 167940,
 'China Cases': 103094,
 'China Deaths': 4846,
 'China Cases (+)': 134,
 'China Deaths (+)': 0,
 'US Cases': 33326437,
 'US Deaths': 596434,
 'US Cases (+)': 71439,
 'US Deaths (+)': 2115,
 'EU Cases': 46766001,
 'EU Deaths': 1074154,
 'EU Cases (+)': 241929,
 'EU Deaths (+)': 5820}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   
Algeria               0        0        0        0        0        0        0   
Andorra               0        0        0        0        0        0        0   
Angola                0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  5/25/21  5/26/21  5/27/21  \
Country/Region                             ...                              
Afghanistan           0        0        0  ...      628      840      623   
Albania               0        0        0  ...       14       15       20   
Algeria               0        0        0  ...      254      285      280   
Andorra               0        0        0  ...       95        7       11   
Angola                0        0        0  ...      310      405      269   

                5/28/21  5/29/21  5/30/21  5/31/21  6/1/21  6/2/21  6/3/21  
Country/Region                                                              
Afghanistan         764      981      650     1077    1139    1049    1093  
Albania              21       12       12        6      22      14       9  
Algeria             272      258      269      188     305     422     336  
Andorra              11        0        0       34       2      15       8  
Angola              337      236      186      185     201     208     180  

[5 rows x 499 columns]

In [9]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)